In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('rule_dataset.csv')

def clean_braces(x):
    x = x.replace("()", "").replace(",", "").replace(" ", "")
    return str(x)
def add_braces(x):
    x = x+"()"
    return x

df['Syscall'] = df['Syscall'].apply(clean_braces)
df.head(3)

# df['Syscall'] = df['Syscall'].apply(add_braces)
# df.to_csv("rule_dataset.csv", index=False)

- 印出各類別的個數和內容

In [ ]:
entityType_lst = df['EntityType'].unique()
lenSyscall = len(df['Syscall'].unique())
lenEnVerb = len(df['EnVerb'].unique())
print(f"For total {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs\n")

for entityType in entityType_lst:
    dataFileType = df[df['EntityType'] == entityType]
    lenEnTp = len(dataFileType)
    lenSyscall = len(dataFileType['Syscall'].unique())
    lenEnVerb = len(dataFileType['EnVerb'].unique())
    actionType_lst = dataFileType['ActionType'].unique()

    print(f"EntityType: {entityType:<6} has {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs & {len(actionType_lst):2} actionType")

    for actionType in actionType_lst:
        dataActionType = dataFileType[dataFileType['ActionType'] == actionType]
        lenSyscall = len(dataActionType['Syscall'].unique())
        lenEnVerb = len(dataActionType['EnVerb'].unique())

        print(f"\t{actionType:<12} has {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs")

    print()

    

試寫 rule
- input: object 和 entityType
- output: 對應的 syscall (1或多個)
- Todo:
    1. 寫規則
    2. 回OPser導出資料
    3. 測試規則
    4. 擴充資料集 (source=report & expend 同義詞)
    5. extract BERT word vector

In [ ]:
df['EntityType'].unique()

In [ ]:
class OperationEvaluator:
    def __init__(self) -> None:
        self.df = self.read_rule()
        self.update_syscall_list = list(self.df[self.df['ActionType'].isin(['UPDATE'])]['Syscall'].unique())
        pass

    def read_rule(self) -> pd.DataFrame:
        df = pd.read_csv('rule_dataset.csv')

        def clean_braces(x):
            x = x.replace("()", "").replace(",", "").replace(" ", "")
            return str(x)

        df['Syscall'] = df['Syscall'].apply(clean_braces)
        return df

    def translate_rule(self, entityType: str, enVerb: str, object: str=None) -> list[str] | None:
        ''' translate a given enverb into system call(s) with hardcoded rule '''
        entityType = entityType.upper()
        if entityType.startswith('PROC'):
            entityType = 'PROC'
        if entityType.startswith('NET'):
            entityType = 'NET'
        if entityType.startswith('INFO'):
            entityType = 'INFO'
        entityType_lst = ['FILE', 'PROC', 'DEVICE', 'INFO', 'NET']
        if entityType not in entityType_lst:
            return None
        
        # Get rules belongs to this entityType
        dataFileType = self.df[self.df['EntityType'] == entityType]
        dataEnVerb = dataFileType.loc[dataFileType['EnVerb'].isin([enVerb])]
        if len(dataEnVerb) == 0:
            return None
        return list(dataEnVerb['Syscall'].unique())

    def resolve(self, entityType: str, syscall:str, en_verb:str, downcast=False) -> bool:
        hit, isSysChange = False, False
        corresponding_syscall_list = self.translate_rule(entityType, en_verb)
        if corresponding_syscall_list and syscall in corresponding_syscall_list:
            hit = True
        elif entityType.upper() == "FILE" and syscall in self.update_syscall_list:
            isSysChange = True
        return hit, isSysChange

# test
oE = OperationEvaluator()
oE.translate_rule('FILE', 'read') # ['read', 'readlink']
ans_syscalls = oE.translate_rule('FILE', 'change') # ['rename', 'fchmod', 'chmod', 'lchmod']
print(ans_syscalls)
ans_syscalls = oE.translate_rule('PROC', 'change') # ['ptrace', 'brk']
print(ans_syscalls)
ans_syscalls = oE.translate_rule('FILE', 'create') # ['dup', 'dup2', 'open', 'mmap', 'link']
print(ans_syscalls)
ans_syscalls = oE.translate_rule('PROC', 'create') # ['clone', 'fork', 'vfork']
print(ans_syscalls)
ans_syscalls = oE.translate_rule('NET', 'create') # ['pipe', 'socket']
print(ans_syscalls)

# test
res = oE.resolve('FILE','linkat', 'read') # (False, False)
print(res) 

In [ ]:
# ! pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
